# SCC0270 - Redes Neurais e Aprendizado Profundo
### Aula 8 - Prática (Redes neurais convolutivas)
**Daniel Penna Chaves Bertazzo - 10349561**

In [ ]:
import tensorflow as tf
import tensorflow.keras as keras
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten
from tensorflow.keras.utils import to_categorical

# Item 1.1 - Preparação dos dados

###  Importação do *dataset* e concatenação em um único conjunto

In [ ]:
# Carrega o dataset
# originalmente ja vem separado em treino e teste, mas aqui vamos juntar tudo
(data, target), (data_aux, target_aux) = tf.keras.datasets.mnist.load_data()

In [ ]:
# Junta as instancias dos dois conjuntos em um unico dataset
data = np.concatenate((data, data_aux))

# Junta os targets dos dois conjuntos em um unico vetor
target = np.concatenate((target, target_aux))

# Para economizar memoria
del data_aux, target_aux

In [ ]:
# Faz o one-hot enconding dos targets (compatibilidade com a saida de uma rede neural)
target = to_categorical(target)

In [ ]:
print(data.shape, target.shape, sep='\n')

### Definição das proporções dos cortes

In [ ]:
# Calcula o tamanho de cada corte -> 10% até 100%
cuts = [int((x/10) * data.shape[0]) for x in range(1, 11)]

### Realização dos cortes

In [ ]:
X = [] # Armazena os 10 conjuntos de instancias
y = [] # Armazena os 10 conjuntos de targets

In [ ]:
for cut in cuts:
    X.append(data[:cut])
    y.append(target[:cut])

### Separação em treino e teste

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
# Armazenam as instancias para treino e teste
X_train = [None] * 10
X_test  = [None] * 10

# Armazenam os targets para treino e teste
y_train = [None] * 10
y_test  = [None] * 10

In [ ]:
for i in range(10):
    X_train[i], X_test[i], y_train[i], y_test[i] = train_test_split(X[i], y[i], test_size=0.3)

### Ajustando as dimensões para ser compatível com o modelo do Keras

In [ ]:
for i in range(10):
    X_train[i] = X_train[i].reshape(X_train[i].shape[0], X_train[i].shape[1], X_train[i].shape[2], 1)
    X_test[i]  = X_test[i].reshape(X_test[i].shape[0], X_test[i].shape[1], X_test[i].shape[2], 1)

#  Item 1.2 - Criação e implementação do modelo
A arquitetura utilizada para todos os cortes será a seguinte:
* **Primeira camada:** convolução com 32 filtros 3x3 e ativação relu + max_pooling 2x2

* **Segunda camada:** convolução com 64 filtros 3x3 e ativação relu + max_pooling 2x2

* **Última camada:** *fully connected* com 10 neurônios (número de classes) e ativação softmax

In [ ]:
model = Sequential()

# Primeira camada (conv + pooling)
# Na primeira camada, é preciso passar o input_shape. Como todos os X_train[i] 
# possuem dados da mesma dimensao, divergindo apenas na quantidade de instancias em cada um,
# pode-se usar o shape de qualquer um deles. Neste caso, usou-se o X_train[0]
model.add(Conv2D(filters=32, kernel_size=(3, 3), activation='relu', input_shape=X_train[0].shape[1:]))
model.add(MaxPooling2D(pool_size=(2, 2)))

# Segunda camada (conv + pooling)
model.add(Conv2D(filters=64, kernel_size=(3, 3), activation='relu', input_shape=X_train[0].shape[1:]))
model.add(MaxPooling2D(pool_size=(2, 2)))

# Transforma os dados para um vetor unidimensional
model.add(Flatten())

# Ultima camada: fully connected com 10 neuronios (1 para cada classe) e
# ativacao softmax para obter as probabilidades
model.add(Dense(10, activation='softmax'))

In [ ]:
model.summary()